In [1]:
from pydantic import BaseModel
import uuid

# Session management testing

In [2]:
# model for session ID
class SessionId(BaseModel):
    session_id: str = "defauld-chat-session"


# session_id = SessionId(session_id = uuid.uuid4().hex).session_id
session_state = set()

# created sesion_id
session_id = SessionId(session_id = uuid.uuid4().hex).session_id

In [3]:
if session_id not in session_state:
    session_state.add(session_id)

print(session_state)

{'2ff2ee1bdcb0435a9a9829e855d066f3'}


# Session management in chatbot

In [4]:
# model for chat request
class ChatRequest(BaseModel):
    message: str

class ChatResponse(BaseModel):
    response: str

In [5]:
# simple chatbot

from groq import Groq
from dotenv import load_dotenv
import os
load_dotenv()

# Initialize the Groq API client using your API key
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

def simple_bot(state: ChatRequest) -> str:

    # if SessionId.session_id not in session_state:
    
    query = state.message
    # Create a simple but context-aware prompt
    prompt = f"Answer the following user query clearly and concisely:\n\n{query}"

    # Send request to Groq model
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    # Extract and return response text
    response_text = response.choices[0].message.content.strip()
    return ChatResponse(response=response_text)


In [6]:
user_query = ChatRequest(message="What is the capital of India?")
user_query

ChatRequest(message='What is the capital of India?')

In [7]:
simple_bot(user_query)

ChatResponse(response='The capital of India is New Delhi.')

# Building a conversational app

In [8]:
import uuid
from typing import Dict
from datetime import datetime
from pydantic import BaseModel
from langchain.memory import ConversationBufferMemory
from langchain.schema import messages_from_dict, messages_to_dict
from groq import Groq
import os
from dotenv import load_dotenv

load_dotenv()

True

### Define models and session storage

In [21]:
# --- Pydantic models ---
class ChatRequest(BaseModel):
    session_id: str = None  # Optional, if not provided, create new
    message: str

class ChatResponse(BaseModel):
    session_id: str
    history: list  # List of {"role": str, "content": str}

# --- Session management ---
# session_store maps session_id -> ConversationBufferMemory
session_store: Dict[str, ConversationBufferMemory] = {}

In [10]:
# groq llm client
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

### Groq wrapper to work with LangChain memory

In [11]:
def groq_chat(messages: list) -> str:

    # if SessionId.session_id not in session_state:
    
    # Create a simple but context-aware prompt
    prompt = f"Answer the following user query clearly and concisely:\n\n{messages}"

    # Send request to Groq model
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    # Extract and return response text
    return response.choices[0].message.content.strip()

In [12]:
groq_chat(messages=["what is the capital of India?"])

'The capital of India is New Delhi.'

In [13]:
import json
CHAT_DIR = "chat_sessions"
os.makedirs(CHAT_DIR, exist_ok=True)

# --- Helper: save session to disk ---
def save_session(session_id: str, memory: ConversationBufferMemory):
    filepath = os.path.join(CHAT_DIR, f"{session_id}.json")
    with open(filepath, "w") as f:
        json.dump([msg.dict() for msg in memory.chat_memory.messages], f)

In [14]:
# --- Helper: load session from disk ---
def load_session(session_id: str) -> ConversationBufferMemory:
    filepath = os.path.join(CHAT_DIR, f"{session_id}.json")
    memory = ConversationBufferMemory(return_messages=True)
    if os.path.exists(filepath):
        with open(filepath, "r") as f:
            messages = json.load(f)
            for msg in messages:
                if msg["type"] == "human":
                    memory.chat_memory.add_user_message(msg["content"])
                else:
                    memory.chat_memory.add_ai_message(msg["content"])
    return memory

In [15]:
# --- Memory Management ---
# In-memory active sessions
active_sessions: Dict[str, ConversationBufferMemory] = {}

In [17]:
session_id = str(uuid.uuid4().hex)
session_id

'c5a7b75a2d5043a68c2d9acfee7b032f'

In [22]:
# --- Chatbot function ---
def chat_bot(request: ChatRequest, save_after_chat: bool = False) -> ChatResponse:
    # --- Load or create session ---
    if not request.session_id:
        session_id = uuid.uuid4().hex
        memory = ConversationBufferMemory(return_messages=True)
    else:
        session_id = request.session_id
        memory = active_sessions.get(session_id)
        if not memory:
            # Load from disk if available
            memory = load_session(session_id)

    # --- Add user message ---
    memory.chat_memory.add_user_message(request.message)

    # --- Prepare messages for Groq ---
    messages_list = [
        {"role": "user" if msg.type == "human" else "assistant", "content": msg.content}
        for msg in memory.chat_memory.messages
    ]

    # --- Get assistant response ---
    assistant_reply = groq_chat(messages_list)
    memory.chat_memory.add_ai_message(assistant_reply)

    # --- Update active sessions ---
    active_sessions[session_id] = memory

    # --- Optionally save session to disk ---
    if save_after_chat:
        save_session(session_id, memory)
        active_sessions.pop(session_id, None)  # remove from active sessions

    return ChatResponse(session_id=session_id, history=messages_list + [{"role": "assistant", "content": assistant_reply}])

In [23]:
# Start new session
req1 = ChatRequest(message="Hello!")
res1 = chat_bot(req1)
print(res1.session_id, res1.history)

e8e2014984a84abcb4a60c8d9937a27a [{'role': 'user', 'content': 'Hello!'}, {'role': 'assistant', 'content': 'Hello. How can I assist you today?'}]


/tmp/ipykernel_74205/2908949389.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


In [24]:
active_sessions

{'e8e2014984a84abcb4a60c8d9937a27a': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Hello!', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello. How can I assist you today?', additional_kwargs={}, response_metadata={})]), return_messages=True)}

In [25]:
active_sessions['e8e2014984a84abcb4a60c8d9937a27a']

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Hello!', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello. How can I assist you today?', additional_kwargs={}, response_metadata={})]), return_messages=True)

### Chatbot function with memory and sessions

In [160]:
def chat_bot(request: ChatRequest) -> ChatResponse:

    # --- 1. Handle session ---
    if not request.session_id:
        # Create new session
        session_id = uuid.uuid4().hex
        memory = ConversationBufferMemory(return_messages=True)
        session_store[session_id] = memory
    else:
        session_id = request.session_id
        memory = session_store.get(session_id)
        if memory is None:
            # Create a new memory if session_id not found
            memory = ConversationBufferMemory(return_messages=True)
            session_store[session_id] = memory

    # --- 2. Add user message to memory ---
    memory.chat_memory.add_user_message(request.message)

    # --- 3. Get full chat history for Groq -
    messages_list = [
    {"role": "user" if msg.type == "human" else "assistant", "content": msg.content}
    for msg in memory.chat_memory.messages
    ]
    # --- 4. Get assistant response ---
    assistant_reply = groq_chat(messages_list)

    # --- 5. Save assistant reply to memory ---
    memory.chat_memory.add_ai_message(assistant_reply)

    # --- 6. Return response with session_id ---
    return ChatResponse(session_id=session_id, history=messages_list + [{"role": "assistant", "content": assistant_reply}])

In [ ]:
# New conversation
req1 = ChatRequest(message="my name is kamal!")
res1 = chat_bot(req1)
print(res1.session_id, res1.history)

15f760738ac543e583a0aedd77766a2b [{'role': 'user', 'content': 'my name is kamal!'}, {'role': 'assistant', 'content': 'Nice to meet you, Kamal. How can I assist you today?'}]


In [178]:
res1.session_id

'15f760738ac543e583a0aedd77766a2b'

In [179]:
res1.history

[{'role': 'user', 'content': 'my name is kamal!'},
 {'role': 'assistant',
  'content': 'Nice to meet you, Kamal. How can I assist you today?'}]

In [172]:
session_store['6a983f763aa840f4a52fd0764edf60ec'].chat_memory.messages[0].content

'Hello!'

In [167]:
session_store['6a983f763aa840f4a52fd0764edf60ec'].chat_memory.messages[1].content

'Hello, how can I assist you today?'